In [ ]:
import networkx as nx
import numpy as np

import network

from preprocess_data import Game, PowerPlay, Situation, read_networks
from datetime import date
from enum import Enum

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from scipy import stats

In [ ]:
def regular_networks():
    for nw in read_networks(Situation.REGULAR):
        assortativity = network.assortativity(nw)
        # time = calculate_time(Game(game), None)
        yield assortativity

In [ ]:
def power_play_networks():
    for nw in read_networks(Situation.POWER_PLAY):
        assortativity = network.assortativity(nw)
        yield assortativity

In [ ]:
def penalty_kill_networks():
    for nw in read_networks(Situation.PENALTY_KILL):
        assortativity = network.assortativity(nw)
        yield assortativity

In [ ]:
# Assortativity: Regular vs Power Play
fig, axs = plt.subplots(2, 1, sharex=True)
fig.set_figheight(10)
fig.set_figwidth(15)

x_reg = list(regular_networks())
x_pp  = list(power_play_networks())
# x_pk  = list(penalty_kill_networks())
n, bins, patches = axs[0].hist(x_reg, facecolor='blue', alpha=0.5)
n, bins, patches = axs[1].hist(x_pp, facecolor='blue', alpha=0.5)
# n, bins, patches = axs[2].hist(x_pk, facecolor='blue', alpha=0.5)
axs[0].set_title("Regular Networks")
axs[1].set_title("Power Play Networks")

plt.savefig("assortativity_rel.png", dpi=300)
plt.show()

print(stats.ttest_ind(x_reg, x_pp, alternative="greater"))
# print(stats.ttest_ind(x_reg, x_pk, alternative="greater"))

In [ ]:
len(x_pp)